In [1]:
import pandas as pd
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [2]:
train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Fare', 'Embarked'], inplace=True) 
train_data = pd.concat([train_data, pd.get_dummies(train_data.Sex)], axis=1).drop(['Sex'], axis=1)
train_data['Cabin'] = train_data['Cabin'].str[0]  # Get the first character of Cabin and One-Hot Encode it
train_data = pd.concat([train_data, pd.get_dummies(train_data.Cabin)], axis=1).drop(['Cabin'], axis=1)
train_data = pd.concat([train_data, pd.get_dummies(train_data.Pclass)], axis=1).drop(['Pclass'], axis=1)
train_data.head(5)

,Survived,Age,SibSp,Parch,female,male,A,B,C,D,E,F,G,T,1,2,3
0,0,22.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,1,38.0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0
2,1,26.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,1,35.0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0
4,0,35.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [4]:
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)  # Fill NaN with mean for age
train_data.fillna(-1, inplace=True)  # -1 for other values
y = train_data['Survived']
x = train_data.iloc[:, 1:]
x.head(5)

,Age,SibSp,Parch,female,male,A,B,C,D,E,F,G,T,1,2,3
0,22.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,38.0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0
2,26.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,35.0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0
4,35.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [5]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))


model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [6]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(x, y, epochs=100, validation_split=0.2)  # For checking validation

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

23/23 [==============================] - 0s 12ms/step - loss: 1.1870 - accuracy: 0.5183 - val_loss: 0.6982 - val_accuracy: 0.6425
Epoch 2/100
23/23 [==============================] - 0s 4ms/step - loss: 0.9360 - accuracy: 0.5028 - val_loss: 0.6823 - val_accuracy: 0.6592
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 0.8408 - accuracy: 0.5562 - val_loss: 0.6446 - val_accuracy: 0.6592
Epoch 4/100
23/23 [==============================] - 0s 4ms/step - loss: 0.7970 - ac

In [7]:
callback2 = tf.keras.callbacks.ModelCheckpoint('weights.h5', monitor='loss', mode='min', save_weights_only=True, save_best_only=True)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x, y, epochs=100, callbacks=[callback2])  # Training

model.load_weights('weights.h5')

Epoch 1/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4183 - accuracy: 0.8215
Epoch 2/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4106 - accuracy: 0.8305
Epoch 3/100
28/28 [==============================] - 0s 3ms/step - loss: 0.3988 - accuracy: 0.8418
Epoch 4/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4034 - accuracy: 0.8373
Epoch 5/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4034 - accuracy: 0.8305
Epoch 6/100
28/28 [==============================] - 0s 3ms/step - loss: 0.3934 - accuracy: 0.8227
Epoch 7/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4051 - accuracy: 0.8283
Epoch 8/100
28/28 [==============================] - 0s 3ms/step - loss: 0.3925 - accuracy: 0.8328
Epoch 9/100
28/28 [==============================] - 0s 3ms/step - loss: 0.3987 - accuracy: 0.8316
Epoch 10/100
28/28 [==============================] - 0s 3ms/step - loss: 0.3954 - accuracy: 0.8305
Epoch 11/

In [8]:
import numpy as np

# Applying the same preprocessing as training
test_data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Fare', 'Embarked'], inplace=True)
test_data = pd.concat([test_data, pd.get_dummies(test_data.Sex)], axis=1).drop(['Sex'], axis=1)
test_data['Cabin'] = test_data['Cabin'].str[0]
test_data = pd.concat([test_data, pd.get_dummies(test_data.Cabin)], axis=1).drop(['Cabin'], axis=1)
test_data['T'] = np.zeros(test_data.shape[0], dtype=int)  # Test Data doesn't have 'T' for cabin, added to meet the feature counts
test_data = pd.concat([test_data, pd.get_dummies(test_data.Pclass)], axis=1).drop(['Pclass'], axis=1)

test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)
test_data.fillna(-1, inplace=True)
test_data.head(5)

,Age,SibSp,Parch,female,male,A,B,C,D,E,F,G,T,1,2,3
0,34.5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,47.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,62.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
3,27.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,22.0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1


In [10]:
predicts = model.predict(test_data)
predicts = np.rint(predicts)
predicts = np.asarray(predicts, dtype=int)
passid = pd.read_csv('test.csv')['PassengerId']

output = pd.DataFrame()
output['PassengerId'] = passid
output['Survived'] = predicts
output.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [11]:
output.to_csv('submission.csv', index=False)
# Score: 0.77033